# Prepare a sample from the BOLT text message corpus for Amazon Mechanical Turk labeling

In [1]:
import os

import pandas as pd
import xmltodict

In [2]:
ltf_dir = '/Users/tmorrill002/Documents/datasets/charm/raw/ltf'
psm_dir = '/Users/tmorrill002/Documents/datasets/charm/raw/psm'

In [3]:
def load_conversation(ltf_file, psm_file, ltf_dir, psm_dir):
    ltf_file = os.path.join(ltf_dir, ltf_file)
    psm_file = os.path.join(psm_dir, psm_file)
    with open(ltf_file, 'r') as f:
        ltf_content = f.read()
    
    try:
        with open(psm_file, 'r') as f:
            psm_content = f.read()
    except FileNotFoundError:
        psm_content = False
    return ltf_content, psm_content

def unpack_attributes(attribute):
    attr_dict = {}
    for attr in attribute:
        attr_dict[attr['@name']] = attr['@value']
    return attr_dict

def merge_metadata(ltf_content, psm_content):
    ltf = xmltodict.parse(ltf_content)
    psm = xmltodict.parse(psm_content)
    
    # filter psm list to message attributes
    message_attr = []
    for d in psm['psm']['string']:
        if d['@type'] == 'message':
            message_attr.append(d)
    
    # unpack the message attributes
    psm_df = pd.DataFrame(message_attr)
    psm_attr_df = pd.DataFrame(psm_df['attribute'].apply(unpack_attributes).values.tolist())
    
    psm_df = pd.concat((psm_df.drop(columns=['attribute']), psm_attr_df), axis=1)
    
    ltf_df = pd.DataFrame(ltf['LCTL_TEXT']['DOC']['TEXT']['SEG'])
    
    # join ltf and psm on start_char
    df = pd.merge(ltf_df, psm_df, left_on='@start_char', right_on='@begin_offset', how='left')
    
    # filter out messages where content length is 0 for a clean inner join
    df = df[df['@char_length'] != '0'].reset_index(drop=True)
    assert (len(df) == len(ltf_df))
    # may still be missing attributes for each message
    return df

In [4]:
ltf_files = [f for f in os.listdir(ltf_dir) if f != '.DS_Store']
psm_files = [f for f in os.listdir(psm_dir) if f != '.DS_Store']

In [5]:
len(ltf_files)

976

In [6]:
len(psm_files)

976

In [7]:
dfs = []
errors = []
for ltf_file in ltf_files:
    psm_file = ltf_file.replace('ltf', 'psm')
    ltf_content, psm_content = load_conversation(ltf_file, psm_file, ltf_dir, psm_dir)
    if psm_content == False:
        errors.append((ltf_file, psm_file, 'PSM file not found'))
        continue
    df = merge_metadata(ltf_content, psm_content)
    
    if len(df[df['participant'].isna()]) > 0:
        errors.append((ltf_file, psm_file, 'Attributes missing'))
        continue
    
    # retain filename
    df.insert(0, 'filename', ltf_file)
    dfs.append(df)

In [8]:
error_df = pd.DataFrame(errors, columns=['ltf_file', 'psm_file', 'error'])

In [9]:
error_df['error'].value_counts()

Attributes missing    2
Name: error, dtype: int64

In [10]:
len(dfs)

974

### Descriptive statistics
- distribution of conversation length
- distribution of utterance length
- distribution of conversation duration (could be useful for breaking conversations up)
- number of speakers
- dist of number of speakers per convo
- dist of number of turns per speaker per convo

In [11]:
df = pd.concat(dfs)
df = df.reset_index(drop=True)

In [12]:
# number of conversations
df['filename'].nunique()

974

In [13]:
# distribution of conversation length
turn_counts = df.groupby('filename').agg(**{'turn_count':('@id', 'count')})

In [14]:
# most conversations are about 82 turns
turn_counts.describe()

,turn_count
count,974.000000
mean,106.039014
std,81.991555
min,7.000000
25%,58.000000
50%,80.000000
75%,125.000000
max,753.000000


In [15]:
# most utterances are about 8 characters
df['ORIGINAL_TEXT'].apply(len).to_frame().describe()

,ORIGINAL_TEXT
count,103282.000000
mean,10.374867
std,10.796316
min,1.000000
25%,4.000000
50%,8.000000
75%,13.000000
max,666.000000


In [16]:
# conversation duration typically about 3 hours
df['time'] = pd.to_datetime(df['time'])
duration_df = df.groupby('filename').agg(**{'start_time': ('time', 'min'), 'end_time': ('time', 'max')})

duration_df['diff'] = duration_df['end_time'] - duration_df['start_time']
duration_df['minutes'] = duration_df['diff'].apply(lambda x: x.seconds / 60)

duration_df['minutes'].describe()

count     974.000000
mean      451.410181
std       512.702620
min         0.000000
25%        50.287500
50%       174.150000
75%       813.462500
max      1439.983333
Name: minutes, dtype: float64

In [17]:
# TODO: investigate natural split points in the conversation for notions of
# intra/inter conversational communcation change points

In [18]:
# fewer speakers than conversations
df['participant'].nunique()

528

In [19]:
# 2 speakers per convo
participants_df = df.groupby('filename').agg(**{'participant_count': ('participant', 'nunique')})
participants_df.describe()

,participant_count
count,974.000000
mean,2.006160
std,0.192252
min,2.000000
25%,2.000000
50%,2.000000
75%,2.000000
max,8.000000


In [20]:
# one conversation with 8 participants
participants_df[participants_df['participant_count'].apply(lambda x: x>2)]

,participant_count
filename,
M01000GNK.ltf.xml,8


In [21]:
# number of turns per speaker per convo
turn_df = df.groupby(['filename', 'participant']).agg(**{'turn_count': ('participant', 'count')})

In [22]:
turn_df = turn_df.groupby('filename').agg(**{'turn_counts':('turn_count', list)})

In [23]:
# remove convos with more than 2 participants
turn_df = turn_df[turn_df['turn_counts'].apply(lambda x: len(x) == 2)]

In [24]:
turn_df['max_pct'] = turn_df['turn_counts'].apply(lambda x: max(x) / sum(x))

In [25]:
turn_df['min_pct'] = 1 - turn_df['max_pct']

In [26]:
# typically one speaker does more of the talking
turn_df.describe()

,max_pct,min_pct
count,973.000000,973.000000
mean,0.579276,0.420724
std,0.066457,0.066457
min,0.500000,0.017544
25%,0.529851,0.390071
50%,0.567568,0.432432
75%,0.609929,0.470149
max,0.982456,0.500000


### Select a representative conversation for annotation

In [27]:
turn_counts[turn_counts['turn_count'] == 82]

,turn_count
filename,
M01000GE6.ltf.xml,82
M01000GEW.ltf.xml,82
M01000GFQ.ltf.xml,82
M01000GH1.ltf.xml,82
M01000GIF.ltf.xml,82
M01000GK0.ltf.xml,82
M01000GOY.ltf.xml,82
M01000GP9.ltf.xml,82
M01000GQK.ltf.xml,82


In [28]:
sample_df = df[df['filename'] == 'M01000GEW.ltf.xml']

In [29]:
sample_df = sample_df[['ORIGINAL_TEXT', 'time', 'participant']]

In [30]:
sample_df = sample_df.rename(columns={'ORIGINAL_TEXT':'Original Text', 'time': 'Time', 'participant': 'Participant'})

In [31]:
sample_df['Participant'] = sample_df['Participant'].apply(lambda x: {'135882': 'A', '138485': 'B'}[x])

In [32]:
# ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.
sample_df['Time'] = sample_df['Time'].dt.tz_localize(None)

In [33]:
sample_df.to_excel('/Users/tmorrill002/Documents/datasets/charm/transformed/sample_M01000GEW.xlsx', index=False)

### Save complete dataset

In [35]:
df.to_csv('/Users/tmorrill002/Documents/datasets/charm/transformed/BOLT_dev_conversations.csv')